## Sorting and Grouping
The Search service offers the ability to organize your result set in different ways.  Whether you choose to sort based on a specific property, group properties or boost the result set, you have the option to override the default ordering returned. In addition, you can also use navigation to organize your buckets of data returned.

By default, the result set will provide some basic ranking, depending on the type of query or filter performed.  

In [1]:
import refinitiv.data as rd
from refinitiv.data.content import search
import pandas as pd

# Default session - desktop
rd.open_session(app_key='Your API Key here')

<refinitiv.data.session.Definition object at 0x1e864631220 {name='default'}>

In [2]:
pd.set_option('display.max_colwidth', 140)
rd.__version__

'1.0.0b9'

#### Example - Default ranking and how to boost results
To demonstrate default sorting, I'm going to search for documents that match a specific ticker. This will result in the documents ranked based on the relevancy and scores assigned to each document.

Search for Vodafone (VOD) against the ticker:

In [3]:
response=search.Definition(
    filter = "TickerSymbol eq 'VOD'",
    select = "_, RCSExchangeCountryLeaf"
).get_data()
response.data.df

,BusinessEntity,DocumentTitle,PermID,PI,RIC,RCSExchangeCountryLeaf
0,QUOTExEQUITY,"Vodafone Group PLC, Ordinary Share, London Stock Exchange",55836053361,1049261,VOD.L,United Kingdom
1,QUOTExEQUITY,"Vodacom Group Ltd, Ordinary Share, Johannesburg Stock Exchange",55849756323,59938825,VODJ.J,South Africa
2,QUOTExEQUITY,"Johannesburg Stock Exchange Vodacom Group Limited Cash Settled SSF Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock ...",21508525903,113268320,VODSc1,South Africa
3,QUOTExEQUITY,"Johannesburg Stock Exchange Can Do VODX Vodacom Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21534036541,148452798,VODXFLc1,South Africa
4,QUOTExEQUITY,"Johannesburg Stock Exchange Vodacom Group CFD Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21600821292,244233208,VODCFDc1,South Africa
5,QUOTExEQUITY,"Johannesburg Stock Exchange Vodacom Group Limited Cash Settled CFD Sabor Equity Future Continuation 1, Equity Future, ZAR, Johannesburg ...",21693141305,363600087,VODSCFDc1,South Africa
6,QUOTExEQUITY,"Johannesburg Stock Exchange Vodacom Group Ltd SSF Cash Settled Dividend Neutral Equity Future Continuation 1, Equity Future, ZAR, Johann...",21707726176,381845837,VODTc1,South Africa
7,QUOTExEQUITY,"Johannesburg Stock Exchange Vodafone Group PLC Dividend Neutral Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exc...",21481060885,53826719,VODDc1,South Africa
8,QUOTExEQUITY,"Johannesburg Stock Exchange Vodafone Group PLC IDX Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21481060918,53826752,VODGc1,South Africa
9,QUOTExEQUITY,"Johannesburg Stock Exchange Vodacom Group SSF Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21481084690,60188651,VODQc1,South Africa


The above output ranked the common share 'VOD.L', listed within the UK, at the top.  This is largely due to the fact that 'VOD.L' is the most significant or liquid asset in the list. However, if I decide to override this default ranking by specifying that I want to show the listed documents within the Unitied States at the top, I can do this 
by applying a *Boost* parameter.

In [4]:
response=search.Definition(
    filter = "TickerSymbol eq 'VOD'",
    boost = "RCSExchangeCountryLeaf eq 'United States'",
    select = "_, RCSExchangeCountryLeaf"
).get_data()
response.data.df

,BusinessEntity,DocumentTitle,PermID,PI,RIC,RCSExchangeCountryLeaf
0,QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, NASDAQ Global Select Consolidated",55839401038,726233,VOD.O,United States
1,QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, NASDAQ Stock Exchange Global Select Market",55835364259,1092283,VOD.OQ,United States
2,QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, Cboe BZX Exchange - Nasdaq Global Select Market",55845638958,53705532,VOD.Z,United States
3,QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, Cboe EDGX Exchange - Nasdaq Global Select Market",21475045362,73238578,VOD.DG,United States
4,QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, Boston SE when trading NASDAQ Global Select Market",55846134804,55957391,VOD.B,United States
5,QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, NYSE Arca when trading NASDAQ Global Select Market",55837254988,754172,VOD.P,United States
6,QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, Cboe BYX Exchange - Nasdaq Global Select Market",21475122433,76383634,VOD.ZY,United States
7,QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, NASDAQ OMX PHLX Global Select Market",21475115114,76271998,VOD.PH,United States
8,QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, Cboe EDGA Exchange - Nasdaq Global Select Market",21475045135,73240457,VOD.DY,United States
9,QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, NYSE National Exchange when trading Nasdaq Global Select Market",55838271132,735992,VOD.C,United States


#### Example - List the youngest CEO's.
The order_by parameters will sort, ascending (default) or descending, based on the birth year property.

Note: In the following example, not every document that identifies a CEO will have a reported birth year. Because we are sorting based on the year they were born, 
all CEO's where the year of birth is not recorded will be bumped to the bottom of the list.

In [5]:
response=search.Definition(
    view = search.SearchViews.PEOPLE,
    filter = "OfficerDirector(RoleTitleCode eq 'CEO' and RoleStatus eq 'Active')",
    order_by = "YearOfBirth desc, LastName, FirstName",
    select = "FullName, YearOfBirth, DTCharacteristics, PrimaryOfficerDirectorRIC, PrimaryOfficerDirector"
).get_data()
response.data.df

,FullName,YearOfBirth,DTCharacteristics,PrimaryOfficerDirectorRIC,PrimaryOfficerDirector
0,Hiroyasu Saito,1996,"Chief Executive Officer, Representative Director",<NA>,Gracia Inc
1,Connor Campbell,1994,"Chief Executive Officer, Co-Founder",<NA>,Osler Diagnostics Ltd
2,Ritesh Agarwal,1993,"Non-Executive Chairman of the Board, Founder",ORAV.NS,Oravel Stays Ltd
3,Yiduitakuya Iwamoto,1993,"Chief Executive Officer, Co-Founder, Representative Director",<NA>,Polyuse Inc
4,Alberto Rizzoli,1993,Chief Executive Officer,<NA>,V7 Ltd
5,Tongda Wu,1993,"Chief Executive Officer, Co-Founder",<NA>,Paifang Technology Tianjin Co Ltd
6,Ross Bailey,1992,"Chief Executive Officer, Founder",<NA>,Appear Here Ltd
7,Christian Besenbruch,1992,Chief Executive Officer and Co-Founder,<NA>,Deep Render Ltd
8,Ed Leon Klinger,1992,Chief Executive Officer,<NA>,Flock Ltd
9,Snigdha Mothukuri,1992,"Chief Executive Officer, Executive Director",JEEV.BO,Jeevan Scientific Technology Ltd


Instead of sorting by the year of birth, let's organize the output so the company is grouped.

**Note**: While we can see the company's grouped, we can also observe the many of the entries do not have a year of birth recorded.

In [6]:
response=search.Definition(
    view = search.SearchViews.PEOPLE,
    filter = "OfficerDirector(RoleTitleCode eq 'CEO' and RoleStatus eq 'Active')",
    group_by = "PrimaryOfficerDirector",
    top = 20,
    select = "FullName, YearOfBirth, DTCharacteristics, PrimaryOfficerDirectorRIC, PrimaryOfficerDirector"
).get_data()
response.data.df

,FullName,YearOfBirth,DTCharacteristics,PrimaryOfficerDirectorRIC,PrimaryOfficerDirector
0,Satya Nadella,<NA>,"Chairman of the Board, Chief Executive Officer",MSFT.O,Microsoft Corp
1,Tim D. Cook,<NA>,"Chief Executive Officer, Director",AAPL.O,Apple Inc
2,Andrew R. Jassy,<NA>,"President, Chief Executive Officer, Director",AMZN.O,Amazon.com Inc
3,Sundar Pichai,1972.0,"Chief Executive Officer, Director",GOOGL.O,Alphabet Inc
4,Mark Zuckerberg,<NA>,"Chairman of the Board, Chief Executive Officer, Founder",FB.O,Meta Platforms Inc
5,Warren E. Buffett,1930.0,"Chairman of the Board, Chief Executive Officer",BRKa,Berkshire Hathaway Inc
6,Daniel Zhang,1972.0,"Chairman of the Board, Chief Executive Officer",BABA.K,Alibaba Group Holding Ltd
7,Fan Jiang,<NA>,President - Taobao and Tmall,BABA.K,Alibaba Group Holding Ltd
8,Huateng Ma,<NA>,"Executive Chairman of the Board, Chief Executive Officer",0700.HK,Tencent Holdings Ltd
9,Al Francis Kelly,<NA>,"Chairman of the Board, Chief Executive Officer",V,Visa Inc


#### Example - Sorting using Navigators
By default, when you use a navigator against a property, it will sort all results based on the number of matches for each value within a bucket.  For example, if I were to list the top 10 exchanges within Canada, we can see the count value ranked, indicating the number instruments matched on that exchange.

In [7]:
response = search.Definition(
    view = search.SearchViews.EQUITY_QUOTES,
    filter = "RCSExchangeCountryLeaf eq 'Canada'",
    top = 0,
    navigators = "ExchangeName(buckets:10)"
).get_data()
response.data.raw["Navigators"]["ExchangeName"]["Buckets"]

[{'Label': 'Montreal Options Exchange', 'Count': 100815},
 {'Label': 'Refinitiv', 'Count': 47932},
 {'Label': 'Montreal Exchange', 'Count': 29815},
 {'Label': 'The Toronto Stock Exchange', 'Count': 11668},
 {'Label': 'Nasdaq CXC', 'Count': 10447},
 {'Label': 'Canadian Securities Exchange/PURE - CSE Listed & Other Canadian Listed Securities',
  'Count': 8553},
 {'Label': 'Canadian Composite Quote/Trade', 'Count': 8491},
 {'Label': 'Tradelogiq Omega ATS', 'Count': 7735},
 {'Label': 'Alpha - Canadian ATS', 'Count': 7473},
 {'Label': 'TSX Venture Exchange (former Canadian Ventures Exchange)',
  'Count': 7305}]

Using the above example, I can instead choose to sort based on the average daily volume within the exchange.  The following search will result in the top 5 Canadian exchanges, ranked based on the 90 day average volume:

In [9]:
response = search.Definition(
    view = search.SearchViews.EQUITY_QUOTES,
    filter = "RCSExchangeCountryLeaf eq 'Canada'",
    top = 0,
    navigators = "ExchangeName(buckets:5, desc:sum_AvgVol90D)"
).get_data()

In [10]:
# Pretty display of the listing
data = []
for exch in response.data.raw["Navigators"]["ExchangeName"]["Buckets"]:
    data.append([exch["Label"], f'{exch["sum_AvgVol90D"]:,}'])

pd.DataFrame(data, columns=["Exchange", "Average 90-day Volume"])

,Exchange,Average 90-day Volume
0,Canadian Composite Quote/Trade,"1,984,526,350.0"
1,The Toronto Stock Exchange,"1,119,394,922.0"
2,TSX Venture Exchange (former Canadian Ventures Exchange),"396,521,414.0"
3,Canadian Securities Exchange - CSE Listed,"122,479,548.0"
4,Canadian Securities Exchange/PURE - CSE Listed & Other Canadian Listed Securities,"49,730,595.0"
